In [41]:
import os, re, time, requests,json
from bs4 import BeautifulSoup
from openai import OpenAI
from datetime import datetime,timedelta
import pandas as pd

# 爬取

## 爬取menet.com

In [2]:
host = 'https://www.menet.com.cn/news'


In [3]:
def get_list(host):
    response = requests.post(host)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = [_ for _ in soup.find_all('h3') if "登录" not in _.text]
    return soup, news_list


In [4]:
def get_info(item, host ='https://www.menet.com.cn'):
    link, title, date = host + item.a['href'], item.text,  item.a['href'].split('/')[3][:8]
    return {
    "title": title,
    'url': link,
    'date':date
}


In [5]:
soup, news_list = get_list(host)


In [6]:
result_menet = list(map(get_info, news_list))


In [7]:
for num, item in enumerate(soup.find_all(lambda tag: tag.has_attr('class') and len(tag)==1 and tag['class'] == ["item_time"])):
    result_menet[num]['date'] = item.text


In [8]:
for _ in result_menet: _['date'] = datetime.strptime(_['date'], '%Y-%m-%d').date()


---

## 爬取pharnexcloud

In [9]:
r = requests.get('https://www.pharnexcloud.com/zixun')


In [10]:
soup = BeautifulSoup(r.text, 'html.parser')


In [11]:
dates = soup.find_all(lambda tag:tag.has_attr('class') and tag['class']==['date'])
dates = [_.text for _ in dates]
dates = [_ for _  in dates if _ != '']
dates = [datetime.strptime(_.split(' ')[0], '%Y-%m-%d').date() for _ in dates]


In [12]:
result = soup.find_all('a', class_ = 'img-content')


In [13]:
def query_tilte_url(item):
    title = item.find_next()['alt']
    url = 'https://www.pharnexcloud.com' + item['href']
    return {
        'title': title,
        'url': url
    }


In [14]:
result_pharnexcloud = list(map(query_tilte_url, result))


In [15]:
for no, item in enumerate(result_pharnexcloud): item['date'] = dates[no]


---

## 爬取bydrug

In [16]:
r = requests.get('https://bydrug.pharmcube.com/')


In [17]:
soup = BeautifulSoup(r.text, 'html.parser')


In [18]:
datas = soup.find_all('div', class_= 'mf-flex mf-items-center')


In [19]:
datas = datas[1:]


In [20]:
item = datas[0]


In [21]:
def get_info(item):
    title = item.text.strip()
    url = item.find('a')['href']
    date = re.findall(r'\d{4}-\d{2}-\d{2}', requests.get(url).text) [0]
    date = datetime.strptime(date, '%Y-%m-%d').date()
    return {
        'title': title,
        'url': url,
        'date': date
    }


In [22]:
result_bydrug = [get_info(item) for _ in datas]


---

In [23]:
## 爬取https://www.businesswire.com/portal/site/home/news/industry/?vnsId=31250

In [24]:
r = requests.get('https://www.businesswire.com/portal/site/home/news/industry/?vnsId=31250')

In [25]:
soup = BeautifulSoup(r.text, 'html.parser')

In [26]:
all = soup.find_all(class_='bwTitleLink')

In [27]:
def get_info(item):
    
    title = item.parent.text
    title = [_ for _ in all[7].parent.text.split('\n') if _ !=''][0]
    url = 'https://www.businesswire.com/' + item.parent.find_all('a')[0]['href']
    date_str = item.parent.find('time').text.strip().split('-')[0].strip()
    date = datetime.strptime(date_str, '%m/%d/%Y') 
    return {
        'title': title,
        'url': url,
        'date': date
    }

In [28]:
result_businesswire = [get_info(_) for _ in all]

In [29]:
results = [*result_menet, *result_pharnexcloud, *result_bydrug, *result_businesswire]

In [31]:
import pandas as pd

In [33]:
df = pd.DataFrame(results)

In [39]:
df['date'] = df['date'].astype('datetime64[ns]')

In [42]:
def get_yesterday():
  
    now = datetime.now()  
      
    # 获取昨天的日期  
    yesterday = now - timedelta(days=1)  
      
    # 将时间设置为0点0时0分  
    yesterday_start = datetime(yesterday.year, yesterday.month, yesterday.day, 0, 0, 0)  
  
    return yesterday_start

In [47]:
df = df.loc[df['date'] >= get_yesterday()].copy()

In [50]:
result_businesswire = df.to_dict(orient='records')

---

In [336]:
def summarize(url):
    context = requests.get('https://r.jina.ai/'+url).text
    url_pattern = re.compile(r'\bhttps?:\/\/[^\s/$.?#].[^\s]*\b')  
    cleaned_text = url_pattern.sub('', context)
    context = cleaned_text[:6000] if len(cleaned_text)>6000  else cleaned_text
    context = "请尽量简洁总结以下内容，并发表你的观点,文字不得超过150字\n"+context.replace("\n","")[:6000]
    client = OpenAI(
    api_key='sk-69c9bf69fa2b4d2985834d6dc17c0790', # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    completion = client.chat.completions.create(
        model="qwen-max", # 更多模型请参见模型列表文档。
        messages=[{"role": "user", "content": context}]
    )
    print(completion.choices[0].message.content)
    
    return completion.choices[0].message.content


In [337]:
for _ in results:
   try:
        _['context'] = summarize(_['url'])
      
   except Exception as e:
       print(e)
   print('------'*20)


新一代特异性溶栓药物重组人尿激酶原在冠脉内应用取得新进展，尤其针对急性ST段抬高型心肌梗死(STEMI)患者，能够有效降低术后“无复流”现象，改善微循环灌注。ERUPTION研究证明其能提升PCI术后血流级别，且不增加出血风险。该药物为中国自主研发，具有较强血栓溶解特异性和安全性，为高血栓负荷STEMI患者提供了更优治疗选项。
------------------------------------------------------------------------------------------------------------------------
18种独家中成药丸剂表现突出，市场规模超270亿，其中心脑血管疾病用药占主导。天士力、达仁堂、雷允上等企业名列前茅，12款新药正蓄势待发。这一趋势显示了中成药市场的强劲增长及企业在创新研发上的积极布局，特别是针对慢性疾病的中药丸剂因其独特优势而需求持续上升。
------------------------------------------------------------------------------------------------------------------------
医药股领域迎来分红热潮，2024年中期超过70家企业公布分红计划，创历史新高。其中，14家企业每股派现5元以上，中药企业尤其亮眼，10家企业分红表现突出。最引人注目的是医药股“一哥”计划近50亿现金分红，东阿阿胶、华东医药等5家企业分红超5亿，展现强劲盈利与回馈股东的决心。高分红比例彰显企业财务健康与对投资者的重视，符合市场期待，促进资本市场健康发展。
------------------------------------------------------------------------------------------------------------------------
第十批国家药品集采或将极度激烈，涉及58个品种和超过550亿市场，多家知名药企如齐鲁、科伦、扬子江、倍特药业等正积极准备。其中，多个品种竞争企业数超过10家，如西格列汀口服常释剂型超过30家竞争。市场关注的大品种包括达格列净、乙酰半胱氨酸吸入剂、哌拉西林钠等，显示出高度竞争和快速增长的市场趋势。药企纷纷通过一致性评价，力求在即

In [338]:
results


[{'title': '新一代特异性溶栓药物重组人尿激酶原冠脉内应用新成果',
  'url': 'https://www.menet.com.cn/info/202409/20240918102202222_148102.shtml',
  'date': datetime.date(2024, 9, 18),
  'context': '新一代特异性溶栓药物重组人尿激酶原在冠脉内应用取得新进展，尤其针对急性ST段抬高型心肌梗死(STEMI)患者，能够有效降低术后“无复流”现象，改善微循环灌注。ERUPTION研究证明其能提升PCI术后血流级别，且不增加出血风险。该药物为中国自主研发，具有较强血栓溶解特异性和安全性，为高血栓负荷STEMI患者提供了更优治疗选项。'},
 {'title': '18个独家中成药丸剂厉害了！31亿大品种领跑，天士力、达仁堂、雷允上上榜，12款新药发力',
  'url': 'https://www.menet.com.cn/info/202409/202409180941284128_148099.shtml',
  'date': datetime.date(2024, 9, 18),
  'context': '18种独家中成药丸剂表现突出，市场规模超270亿，其中心脑血管疾病用药占主导。天士力、达仁堂、雷允上等企业名列前茅，12款新药正蓄势待发。这一趋势显示了中成药市场的强劲增长及企业在创新研发上的积极布局，特别是针对慢性疾病的中药丸剂因其独特优势而需求持续上升。'},
 {'title': '医药股“一哥”近50亿现金分红太惊人！东阿阿胶、华东……超5亿，10家中药企业亮眼',
  'url': 'https://www.menet.com.cn/info/202409/202409180938503850_148097.shtml',
  'date': datetime.date(2024, 9, 18),
  'context': '医药股领域迎来分红热潮，2024年中期超过70家企业公布分红计划，创历史新高。其中，14家企业每股派现5元以上，中药企业尤其亮眼，10家企业分红表现突出。最引人注目的是医药股“一哥”计划近50亿现金分红，东阿阿胶、华东医药等5家企业分红超5亿，展现强劲盈利与回馈股东的决心。高分红比例彰显企业

In [339]:
def get_flag(item):
    
    title = item['title']
    url = item['url']
    date = item['date'].strftime('%Y-%m-%d')
    context = item['context']
    return f"""
  <h4 style="color:black; margin-top:10px">  
    {title}  
  </h4>  
  
  <span> {date}</span>  
  <div  style="margin-bottom: 20px;">  
    <p>  
      {context}  
    </p>  
    <span><a href={url}>👉查看更多内容</a></span>  
  </div>  
  <hr />  
  
  
""".strip()


In [340]:
htmls = []
for item in results:
    try:
        htmls.append(get_flag(item))
    except:
        pass

In [341]:
htmls = "".join(htmls)

In [342]:
def get_token():
    corp_id = 'ww9bffa3285527b3c8'
    secret = '_uOqAIKUWnRgFIY23gItQ5fX4OeWRpYu1G86Gvk7EUw'
    url = f'https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={corp_id}&corpsecret={secret}'
    return requests.get(url).json()['access_token']


In [343]:
try:
    html_text = f"""  
<!DOCTYPE html>  
<html lang="en">  
  
<head>  
  <meta charset="UTF-8">  
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">  
  <title>Document</title>  
</head>  
<style>  
  /* 全局样式 */  
  * {{  
    margin: 0;  
    padding: 0;  
  }}  
  
  h4 {{  
    text-align: center;  
     text-shadow: 2px 2px 4px rgba(15, 47, 206, 0.5);
  }}  
  
  em {{  
    font-family: 'Franklin Gothic Medium', 'Arial Narrow', Arial, sans-serif;  
    color: silver;  
  }}  
  
  div > p {{  
    width: 96%;  
    padding: 2%;  
  }}  
  
  a {{  
    color: #1e90ff;  
    text-decoration: none;  
    transition: color 0.3s;  
  }}  
  
  a:hover {{  
    color: #ff6347;  
  }}  
  
  p {{  
    text-indent: 2em;  
    text-align: justify;  
  }}  

  div {{
      margin-bottom: 10px;
  }}
</style>  
  
<body>  
  <h4 style="color:pink;">  
    {title}  
  </h4>  
  
  <span> 🗓️ {date}</span>  
  <div>  
    <p>  
      {context}  
    </p>  
    <span><a href={url}>👉查看更多内容</a></span>  
  </div>  
  <hr />  
  
</body>  
  
</html>  
"""
except:
    pass


In [344]:
first = f"""  
<!DOCTYPE html>  
<html lang="en">  
  
<head>  
  <meta charset="UTF-8">  
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">  
  <title>Document</title>  
</head>  
<style>  
  /* 全局样式 */  
  * {{  
    margin: 0;  
    padding: 0;  
  }}  
  
  h4 {{  
    text-align: center;  
  }}  
  
  em {{  
    font-family: 'Franklin Gothic Medium', 'Arial Narrow', Arial, sans-serif;  
    color: silver;  
  }}  
  
  div > p {{  
    width: 96%;  
    padding: 2%;  
  }}  
  
  a {{  
    color: #1e90ff;  
    text-decoration: none;  
    transition: color 0.3s;  
  }}  
  
  a:hover {{  
    color: #ff6347;  
  }}  
  
  p {{  
    text-indent: 2em;  
    text-align: justify;  
  }}  
</style>  
  
<body>  
""".strip()


In [345]:
last = """</body>  
  
</html>  
""".strip()


In [346]:
html_text = first+htmls+last


In [347]:
html_text = html_text.replace('\n', '')


In [348]:
data = {
   "touser" : "@all",
   "toparty" : "PartyID1|PartyID2",
   "totag" : "TagID1 | TagID2",
   "msgtype" : "text",
   "agentid" : '1000002',
   "text" : {
       "content" : html_text
   },
   "safe":0,
   "enable_id_trans": 0,
   "enable_duplicate_check": 0,
   "duplicate_check_interval": 1800
}


In [349]:
from datetime import datetime


In [350]:
title  = datetime.now().strftime("%Y年%m月%d日") + '医疗行业资讯'


In [351]:
data = {
   "touser" : "chenweishi|SuBei|liqian9|YuanGuangJi", # 切换正式环境必须修改！
   
   "msgtype" : "mpnews",
   "agentid" : 1000002,
   "mpnews" : {
       "articles":[
           {
               "title": title, 
               "thumb_media_id": "2tNI-yo4L7-cQmsgW2drOBeFz51E_R5vHa_mdIdU2pBdFUEPklOjTTM_SevJplbmb4khjjhmEVh_kZDALSuo8aQ",
               "author": "大倩倩",
               
               "content": html_text,
               "digest": " "
            }
       ]
   },
   "safe":0,
   "enable_id_trans": 0,
   "enable_duplicate_check": 0,
   "duplicate_check_interval": 1800
}


In [352]:
requests.post("https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token="+get_token(), data= json.dumps(data), ).json()


{'errcode': 0,
 'errmsg': 'ok',
 'msgid': 'mrVtVXE39it1tWVvd57npAtW28DbYkwbbMkhyLcMimAyb7ilc0E9i20hvROUtZWvoMy_v5r10oJPSSTuDi5wew'}